Collab second part
https://colab.research.google.com/drive/1ke3E8dlZqdTh0ftR6uKX-RoevjyTXFta#scrollTo=PXSquJRRIQpj&uniqifier=1



In [ ]:
from google.colab import drive
import csv
import pandas as pd
import numpy as np
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Loading data

In [ ]:
# read the csv data file
df = pd.read_csv('/content/drive/My Drive/You_trash/bgg-13m-reviews.csv',index_col=0)

# select particular columns
bgg_r = df[["user","rating","ID", "comment"]]  # TODO: add 'comments' but when using .dropna() in the next cell don't consider comments please
del df

# rename game ID column
bgg_r.rename(columns={'ID': 'gameId'}, inplace=True)

# drop rows with null values
bgg_r.dropna(inplace=True, subset=['user', 'rating', 'gameId'])

# create new ids for users and games (this is used for building the sparse matrix)
user_c = pd.Categorical(bgg_r.user)
game_c = pd.Categorical(bgg_r.gameId)

# print number of users and games
n_games = game_c.categories.size
n_users = user_c.categories.size
print('number of users:', n_users)
print('number of games:', n_games)

# add new id rows
bgg_r['userIdx'] = user_c.codes
bgg_r['gameIdx'] = game_c.codes
bgg_r  # 13170007 rows, 6 columns


/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,user,rating,gameId,comment,userIdx,gameIdx
0,sidehacker,10.0,13,NaN,261708,12
1,Varthlokkur,10.0,13,NaN,133278,12
2,dougthonus,10.0,13,"Currently, this sits on my list as my favorite...",176672,12
3,cypar7,10.0,13,"I know it says how many plays, but many, many ...",169275,12
4,ssmooth,10.0,13,NaN,266601,12
...,...,...,...,...,...,...
3356495,pswissler,3.0,8256,NaN,248216,4219
3356496,Sprayoncrayon,3.0,8256,"Lego created a version of Pictionary, only you...",118573,4219
3356497,PhoenixSong,2.5,8256,This game is very similar to Creationary. It c...,97862,4219
3356498,tsantos,2.0,8256,This game was really bad. Worst that I've pla...,277620,4219


## Build user-item matrix (sparse matrix)

In [ ]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

# build user-item sparse matrix
ui_sparse_matrix = csr_matrix((bgg_r["rating"], (bgg_r['userIdx'], bgg_r['gameIdx'])), shape=(n_users, n_games))

# build dataframe from user-item sparse matrix
df_ui_matrix = pd.DataFrame.sparse.from_spmatrix(ui_sparse_matrix)
del ui_sparse_matrix
df_ui_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,17025,17026,17027,17028,17029,17030,17031,17032,17033,17034,17035,17036,17037,17038,17039,17040,17041,17042,17043,17044,17045,17046,17047,17048,17049,17050,17051,17052,17053,17054,17055,17056,17057,17058,17059,17060,17061,17062,17063,17064
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289908,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
289909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
289910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
289911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,9.0,8.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Preprocessing the text

We shall preprocess comments given by the users and feed them into a Word2Vec model. The end result of this part is a list of all sentences. We have pickled the `sentences` list, so if available, go ahead to the cell which will load sentences directly to save processing time.

In [ ]:
n_comments = bgg_r['comment'].count()
print('Number of comments:', n_comments)

raw_corpus = bgg_r['comment'].dropna().str.cat(sep=' ')
print(f'length of corpus in chars: {len(raw_corpus):,}')

Number of comments: 2637756
length of corpus in chars: 563,106,169


In [ ]:
# import natural language toolkit
import nltk

# download the punkt tokenizer
nltk.download('punkt')

# Load the punkt tokenizer
tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")

# we tokenize the raw string into raw sentences
raw_sentences = tokenizer.tokenize(raw_corpus)

# delete the raw corpus, we no longer need it
del raw_corpus

print("We have {0:,} raw sentences".format(len(raw_sentences)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


NameError: ignored

In [ ]:
%%time
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm
nltk.download('stopwords')


class Preprocessor():
    strip_special_chars = re.compile("[^A-Za-z]+")
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))

    def preprocess(self, raw_text):
        raw_text = raw_text.lower()
        return self._clean_and_split_str(raw_text)

    # Clean and split text into words
    def _clean_and_split_str(self, string):
        string = re.sub(self.strip_special_chars, " ", string)
        tokens = word_tokenize(string)
        stemmer = self.stemmer
        stop_words = self.stop_words
        # apply stemmer and remove stopwords
        tokens = [stemmer.stem(i) for i in tokens if not i in stop_words]
        return tokens


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
CPU times: user 111 ms, sys: 5.07 ms, total: 116 ms
Wall time: 202 ms


In [ ]:
# clean each raw sentences and build the list of sentences
preprocess = Preprocessor().preprocess
sentences = []
for raw_sent in tqdm(raw_sentences, miniters=1000):
    if len(raw_sent) > 0:
        sentences.append(preprocess(raw_sent))

del raw_sentences
print("We have {0:,} clean sentences".format(len(sentences)))

In [ ]:
# save sentenses for later use
# import pickle
# sentences_path = '/content/drive/My Drive/You_trash/sentenses.p'
# with open(sentences_path, 'wb') as f:
#     pickle.dump(sentences,f)

### Loading sentences
Run this cell to load sentences if you have `sentences.p` file saved and you want to skip creating it again.

In [ ]:
%%time
import pickle
# load sentences
with open(sentences_path, 'rb') as f:
    sentences = pickle.load(f)

print('loaded sentences')
print('count:', len(sentences))  # 7089851

loaded sentences
count: 7089851
CPU times: user 21.7 s, sys: 4.8 s, total: 26.5 s
Wall time: 29.8 s


In [ ]:
token_count = sum([len(sentence) for sentence in sentences])
print("The dataset corpus contains {0:,} tokens".format(token_count))

# number of tokens reduced by x2 after applying stemming and preprocessing, so 54,273,206

The dataset corpus contains 54,273,206 tokens


## Training Word2Vec on the centences

The end result of this section is to train a Word2Vec model and pickle its word vectors (`.wv` attribute) to a file. So if you already have the pickled file you can skip the training process and proceed by loading the word vectors from disk.

In [ ]:
import multiprocessing
 
#Dimensionality of the resulting word vectors
num_features = 300
 
#Minimum word count threshold
min_word_count = 3
 
#Number of threads to run in parallel
num_workers = multiprocessing.cpu_count()
 
#Context window length
context_size = 5
 
#Seed for the RNG, to make the result reproducible
seed = 1

In [ ]:
%%time
import gensim

word2vec_model = gensim.models.word2vec.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers, 
    size=num_features, 
    min_count=min_word_count, 
    window=context_size)

word2vec_model.build_vocab(sentences=sentences)
print("The vocabulary is built")
print("Word2Vec vocabulary length: ", len(word2vec_model.wv.vocab))
 
# Start training the model
word2vec_model.train(sentences=sentences, total_examples=word2vec_model.corpus_count, epochs=5)
print("Training finished")

# delete sentences, we no longer need them
del sentences

The vocabulary is built
Word2Vec vocabulary length:  94297
Training finished
CPU times: user 53min 49s, sys: 4.69 s, total: 53min 54s
Wall time: 14min 15s


In [ ]:
# Save the model
# w2v_vectors_path = f"/content/drive/My Drive/You_trash/w2v_vectors_{num_features}.w2v"
# word2vec_model.wv.save(w2v_vectors_path)
# print("Word vectors saved to", w2v_vectors_path)

### Load pickled Word2Vec model
Run this cell if you have the pickled model file in your disk.

In [ ]:
# Load our word2vec model
from gensim.models import KeyedVectors
wv = KeyedVectors.load(w2v_vectors_path, mmap='r')
print("Word vectors loaded")
print('vector size:', wv.vector_size)  # 94297
print('vocabulary size:', len(wv.vocab))

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Word vectors loaded
vector size: 300
vocabulary size: 94297


## Create item features using word2vec vectors

In [ ]:
game_comments = bgg_r.groupby('gameIdx').apply(lambda df: df['comment'].str.cat(sep=' '))
game_comments

gameIdx
0        Play this game.  Play this game a lot. hard to...
1        Among my favorite games to play with 4 players...
2        Variable winning conditions make for great int...
3        Great game!!!  Tried with 2 players first.  Th...
4        Classic game of Finance and Risk. one of the b...
                               ...                        
17060    Very smart and surprisingly deep tile laying g...
17061    Pure fun Surprisingly good dexterity game with...
17062    If you prefer dice to cards, the news is good....
17063    Finally!  The Firefly board game we all wanted...
17064    Kickstarter backer. Wow there are a lot of ign...
Length: 17065, dtype: object

In [ ]:
def encode_text(vw, preprocessor, text):
    ''' Encode the text using the word2vec word vectors,
    by averaging across all words contained in the text
    '''
    # do the preprocessing and get tokens
    tokens = preprocessor.preprocess(text)
    vector_size = vw.vector_size
    vector = np.zeros(vector_size)
    for token in tokens:
        try:
            vector += vw[token]
        except KeyError:
            pass
    vector = vector/len(tokens)
    return vector

preprocessor = Preprocessor()
game_features = np.empty((n_games, num_features))
for gameIdx, comment in tqdm(game_comments.iteritems(), total=len(game_comments)):
    vector = encode_text(wv, preprocessor, comment)
    game_features[gameIdx] = vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


In [ ]:
# save game comment features to a file
game_comment_features_path = '/content/drive/My Drive/You_trash/game_comment_features.npy'
with open(game_comment_features_path, 'wb') as f:
    np.save(f, game_features)

In [ ]:
# load game features
with open(game_comment_features_path, 'rb') as f:
    game_features = np.load(f, allow_pickle=True)

print(game_features.shape)